In [ ]:
import vispy
import vispy.visuals.transforms as transforms
import vispy.scene.visuals as visuals
import vispy.plot as vp
import numpy as np
import phased_array

vispy.use("jupyter_rfb")

Assumes embedded element pattern is an omnidirectional antenna. Uses equations
in Phased Array Antenna Handbook, 3rd Edition.




$$
F(θ, ϕ) = \sum a_i \exp(jk \pmb{r}_i \cdot \pmb{\hat{r}})
$$
        where

$$
\begin{align*}
k         &= 2 \frac{π}{λ} & \text {wave number} \\
\pmb{\hat{r}}   &= \pmb{\hat{x}} u_0 + \pmb{\hat{y}} + v_0 \pmb{\hat{r}}  + \cos θ_0 & \text{direction of oncoming wave} \\
\pmb{r}_i &= \pmb{\hat{x}} x_i + \pmb{\hat{y}} y_i + \pmb{\hat{z}} z_i & \text{ the position of the $i$dh element} \\
u         &= \sin {θ} \cos {ϕ}  & \text{direction cosine $u$} \\
v         &= \sin{θ} \sin{ϕ} & \text{direction cosine $v$} \\
\\
θ         &= \sin^{-1}(\sqrt{u^2 + v^2}) \\
ϕ         &= \tan^{-1}(\frac{v}{u})
\end{align*}
$$



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
d = 1.0
n = 8
λ = 2.0
a_i = np.ones(n)
arr = phased_array.PhasedArray.ula(d, n)
x = np.linspace(-90, 90, 300)
θ = np.radians(x)
ϕ = np.zeros_like(θ)
af = arr.array_factor(λ, a_i, θ, ϕ)

fig = vp.Fig()
plot = fig[0, 0]
af_db = 20 * np.log10(np.abs(af))
af_db -= np.max(af_db)
plot.plot((x, af_db), marker_size=0.0, color="blue", width=2.0)
plot.camera.set_range(y=[-40, 0])

# fig

In [ ]:
import vispy.scene.visuals as visuals
import ipywidgets as ipw

n = 200

class PlanarArrayVisualization:
    def __init__(self):
        self._last_cartesian = None
        self._layout()
        self.af()

    def af(self):
        dx, dy = self.dx.value, self.dy.value
        nx, ny = self.nx.value, self.ny.value
        λ = self.λ.value
        dynamic_range = self.dynamic_range.value
        self.array = phased_array.PhasedArray.planar(dx, dy, nx, ny)

        u = np.linspace(-1.0, 1.0, n)
        v = np.linspace(-1.0, 1.0, n)
        uu, vv = np.meshgrid(u, v)
        mag = uu**2 + vv**2
        # where we are measuring array factor (everywhere)
        θ, ϕ = phased_array.uv_to_θϕ(uu, vv)
        # where we are pointing
        point_θ, point_ϕ = np.deg2rad(self.θ.value), np.deg2rad(self.ϕ.value)
        a_i = self.array.weights_at_θϕ(λ, point_θ, point_ϕ)
        af = self.array.array_factor(λ, a_i, θ, ϕ)
        self.af_db = af_db = 20 * np.log10(np.abs(af))
        self.af_db_norm = af_db_norm = af_db - np.nanmax(af_db)
        af_db_norm[af_db_norm < -dynamic_range] = -dynamic_range
        cartesian = self.cartesian.value
        max_af = 20 * np.log10(nx * ny)
        if cartesian:
            # scale things based on the maximum possible array factor value
            x, y, z = phased_array.θφr_to_xyz(θ, ϕ, af_db)
            z[z <= 0] = np.nan
            self.surf.set_data(z=z, x=x, y=y)
            self.surf.transform = transforms.STTransform((1, 1, 1))
            self.xax.transform = transforms.STTransform(scale=(max_af, 1, 1), translate=((0, 0, 0)))
            self.yax.transform = transforms.STTransform(scale=(1, max_af, 1), translate=((0, 0, 0)))
            if self._last_cartesian != self.cartesian.value:
                # need to reset the transform to default
                self.plot.camera.scale_factor = dynamic_range
                self.plot.camera.center = (0, 0, dynamic_range / 2)
                self.plot.camera.azimuth = -45
                self.xax.domain = self.yax.domain = (-max_af, max_af)
                self.xax.axis_label = "x"
                self.yax.axis_label = "x"
        else:
            # normalize
            stretch = max_af / 2.5
            self.surf.set_data(z=af_db_norm, x=uu, y=vv)
            self.xax.transform = transforms.STTransform(scale=(stretch, 1, 1), translate=((0, -stretch, -dynamic_range)))
            self.yax.transform = transforms.STTransform(scale=(1, stretch, 1), translate=((-stretch, 0, -dynamic_range)))
            self.surf.transform = transforms.STTransform(
                (stretch, stretch, 1)
            )
            if self._last_cartesian != self.cartesian.value:
                self.plot.camera.scale_factor = dynamic_range * 1.1
                self.plot.camera.center = [0, 0, (-dynamic_range * 1.3)/ 2]
                self.plot.camera.azimuth = -45
                self.xax.domain = (-1, 1)
                self.yax.domain = (-1, 1)
                self.xax.axis_label = "u"
                self.yax.axis_label = "v"
                
        self.plot.camera.fov = 0
        cmap = vispy.color.get_colormap("viridis")
        colors_idx = np.zeros(af_db.shape + (3,))
        # scale between 0 to 1
        colors_idx = (af_db_norm + dynamic_range) / dynamic_range
        colors = cmap[colors_idx.ravel()]
        self.surf.set_data(colors=colors)
        # shading: None, "flat", or "smooth"
        self.surf.shading = None
        self._last_cartesian = cartesian

    def _af(self, _):
        self.af()

    def _layout(self):
        self.fig = vp.Fig()
        self.plot = self.fig[0, 0]
        self.surf = self.plot.surface(np.array([[0, 1], [0, 1]]))

        self.xax = visuals.Axis(pos=[[-1, 0], [1, 0]], domain=(-1, 1), axis_color='k', tick_color='k', text_color='k', parent=self.plot.view.scene, tick_font_size=20)
        self.yax = visuals.Axis(pos=[[0, -1], [0, 1]], domain=(-1, 1), axis_color='k', tick_color='k', text_color='k', parent=self.plot.view.scene, tick_font_size=20)

        self.nx = ipw.BoundedIntText(min=1, max=1000, value=8, step=1, description="nx")
        self.ny = ipw.BoundedIntText(min=1, max=1000, value=8, step=1, description="ny")
        self.dx = ipw.BoundedFloatText(
            min=0.001, max=1, value=0.02, step=0.001, description="dx"
        )
        self.dy = ipw.BoundedFloatText(
            min=0.001, max=1, value=0.02, step=0.001, description="dy"
        )
        self.λ = ipw.BoundedFloatText(
            min=0.001, max=1.0, value=0.04, step=0.001, description="λ"
        )
        self.θ = ipw.FloatSlider(min=0, max=90, value=0, step=0.1, description="θ")
        self.ϕ = ipw.FloatSlider(
            min=-360, max=360, value=0, step=0.1, description="ϕ"
        )
        self.dynamic_range = ipw.FloatSlider(min=10, max=100, step=0.1, value=40, description="Dynamic Range (dB)")
        self.cartesian = ipw.Checkbox(description="Cartesian", value=False)
        self.vbox = ipw.VBox(
            children=[
                self.λ,
                self.θ,
                self.ϕ,
                self.nx,
                self.ny,
                self.dx,
                self.dy,
                self.dynamic_range,
                self.cartesian,
                self.fig.native,
            ]
        )
        for widget in self.vbox.children:
            widget.observe(self._af, names="value")

In [ ]:
vis = PlanarArrayVisualization()
vis
vis.vbox